# Bibliotecas

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np
import faiss

Problema

Temos textos com breves descrições de animais e metadados mostrando informações de cobertura (pelo, penas, pele, escamas), e queremos encontrar, destes, os felinos

## Dados Base

Nos dados de exemplo, temos 5 felinos dentre 20 animais distintos.

In [ ]:
info_animais = [
    "Leões são quadrúpedes musculosos com pelagem curta e dourada, têm presas afiadas e caçam em grupos na savana africana.",
    "Tigres são felinos solitários e robustos, quadrúpedes de pelagem listrada e presas poderosas, que habitam florestas tropicais e pântanos.",
    "Guepardos são quadrúpedes esguios com pelagem amarelada pontilhada de manchas, garras semi-retráteis e corpo adaptado à corrida em planícies abertas.",
    "Leopardos são felinos quadrúpedes ágeis com pelagem amarelada estampada por rosetas, arrastam presas para galhos de árvores e têm presas retráteis.",
    "Gatos domésticos são quadrúpedes pequenos, de pelagens variadas e macias, com dentes caninos afiados e comportamento muitas vezes noturno.",
    "Cachorros são quadrúpedes domesticados, exibem pelagem que varia de curta a longa, têm dentes caninos pronunciados e são altamente sociais.",
    "Elefantes são mamíferos quadrúpedes enormes, com pele espessa quase sem pelos, presas de marfim e tromba versátil, movem-se entre terra e poças d'água.",
    "Cavalos são quadrúpedes elegantes de pelagem curta e crina densa, cascos resistentes e dentes molares fortes, usados para montaria e tração.",
    "Girafas são quadrúpedes extremamente altos, com manchas de pelagem que ajudam na camuflagem e longos pescoços, pastam em copas de árvores baixas.",
    "Zebras são quadrúpedes com pelagem listrada em preto e branco, pelagem curta, cascos duros e vivem em bandos nas planícies africanas.",
    "Rinocerontes são quadrúpedes corpulentos com pele grossa e um ou dois chifres de queratina, pastam em savanas e florestas tropicais.",
    "Hipopótamos são quadrúpedes semi-aquáticos de pele lisa e grossa, com presas fortes, passam grande parte do dia submersos em rios e lagos.",
    "Ursos são quadrúpedes robustos, com pelagem densa, garras afiadas e dentes caninos grandes, percorrendo florestas e montanhas.",
    "Crocodilos são répteis quadrúpedes de pele escamosa, mandíbulas poderosas repletas de dentes pontiagudos, emboscam presas próximas à água.",
    "Golfinhos são mamíferos aquáticos de corpo hidrodinâmico, sem pelos, com nadadeiras e dentes cônicos, nadam em grupos altamente sociais.",
    "Baleias-azuis são mamíferos marinhos gigantes, sem pelos, com corpo alongado e nadadeiras largas, filtram krill com suas barbatanas.",
    "Tubarões-brancos são peixes cartilaginosos de pele áspera, corpo fusiforme e fileiras de dentes triangulares afiados, caçam em mar aberto.",
    "Águias são aves de rapina bípedes, com asas largas e penas resistentes, bico curvo forte e garras afiadas, caçam em grandes altitudes.",
    "Pinguins-imperadores são aves aquáticas bípedes, com plumagem densa e oleosa, asas transformadas em nadadeiras e bico resistente, vivem no gelo.",
    "Papagaios são aves tropicais bípedes, com plumagem colorida, bico curvo forte e patas com dedos opositores, imitam sons do ambiente.",
]

In [ ]:
metadados_animais = [
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pele"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pele"},
    {"cobertura": "pele"},
    {"cobertura": "pelo"},
    {"cobertura": "escamas"},
    {"cobertura": "pele"},
    {"cobertura": "pele"},
    {"cobertura": "pele"},
    {"cobertura": "penas"},
    {"cobertura": "penas"},
    {"cobertura": "penas"},
    {"cobertura": "penas"},
]

# Solução (Busca Semântica Vetorial)

## Carregando Modelos

In [ ]:
# Escolhido por funcionar bem com português e ser pequeno
modelo = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Modelo pequeno e rápido de encoding cruzado
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

## Construindo Banco Vetorial

### Criando Embeddings

In [ ]:
# Transformando os documentos dos animais em embeddings
embeddings = modelo.encode(info_animais, convert_to_numpy=True)

In [ ]:
# Normalizando os embeddings
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
print(embeddings)  # Cada embedding é um vetor de 384 dimensões

[[ 3.3081710e-02  3.3173956e-02 -1.9486861e-03 ...  8.2796738e-02
   3.4752913e-02 -5.5807233e-03]
 [-1.0931087e-02  3.9191678e-02 -3.6326209e-03 ...  7.0279770e-02
   2.4959791e-02 -5.8156073e-02]
 [-2.6445818e-05  2.6257550e-02 -4.8183431e-03 ...  7.5843804e-02
   4.3294370e-02  5.4644980e-03]
 ...
 [-2.4663955e-02 -4.4664595e-02 -3.2132873e-03 ...  4.9569897e-02
   3.2282695e-02 -7.5200815e-03]
 [ 1.7100263e-03 -2.4201316e-03 -4.6152845e-03 ...  6.0225263e-02
   2.6223931e-02 -7.9301819e-03]
 [-2.0529827e-02  1.3451318e-02 -4.5117000e-03 ...  6.0779121e-02
   1.0727278e-02 -5.8815018e-03]]


### Criando Índice no FAISS

In [ ]:
dimensoes = embeddings.shape[1]
index = faiss.IndexFlatIP(dimensoes)
index.add(embeddings)

In [ ]:
# Cada item no index é um vetor numérico:
print(index.reconstruct(0))

[ 3.30817103e-02  3.31739560e-02 -1.94868608e-03  2.69418489e-02
  3.32726426e-02  1.67922098e-02 -2.83533782e-02 -1.81871727e-02
 -3.93361561e-02 -8.90237372e-03 -3.62236723e-02 -2.38379464e-03
  2.38257069e-02  1.27662905e-02  3.89143601e-02 -2.85061337e-02
  3.08176279e-02  6.73467219e-02 -1.60787366e-02  4.72269021e-03
 -3.40103079e-03 -4.77937842e-03  1.21863438e-02 -4.90128761e-03
 -4.96988371e-02 -1.59404166e-02  9.34124645e-03  4.43993509e-02
  9.23475064e-03 -6.76071197e-02  2.68333107e-02 -4.78714816e-02
  1.68640204e-02  3.77251506e-02 -4.40353807e-03 -2.42195427e-02
 -1.83507483e-02 -2.09418871e-02 -1.44138047e-02  1.50539735e-02
  8.00272748e-02 -3.33820768e-02  1.24344388e-02 -1.61631592e-02
 -1.08302645e-02 -1.49948031e-01  1.94011237e-02 -7.73409083e-02
  5.46670221e-02 -4.05859761e-03  3.70441703e-03 -8.11741315e-03
  2.72978516e-03  2.15713810e-02 -4.95272875e-02 -7.79758915e-02
 -2.48870943e-02 -4.86634448e-02 -3.15146819e-02  6.72613308e-02
 -1.06978277e-02 -4.22491

## Consulta no Banco

### Consulta Inicial

In [ ]:
# O que quero encontrar no banco?
query = "Felinos nativos e seus detalhes"

In [ ]:
# Criando Embedding e Normalizando a Query
query_embedding = modelo.encode([query], convert_to_numpy=True)
query_embedding = query_embedding / np.linalg.norm(
    query_embedding, axis=1, keepdims=True
)

In [ ]:
print(query_embedding)

[[-1.46559710e-02  8.42298046e-02 -5.26088756e-03  1.09201418e-02
   4.03903723e-02  1.33562572e-02 -1.76421367e-02  1.29161859e-02
   2.03019809e-02  2.11988133e-03 -2.61760112e-02  3.25233536e-03
   4.85586375e-02 -2.72565279e-02  4.01805155e-02 -7.98555687e-02
   2.39536613e-02  6.50309324e-02 -8.75246990e-03  3.15177371e-03
  -3.19390222e-02 -1.11268768e-02 -2.93883402e-03  5.94953960e-03
  -8.62585381e-02 -3.62075567e-02 -3.19848321e-02  5.87367006e-02
   2.43340153e-02 -5.30066118e-02  4.23265770e-02 -3.68584357e-02
   2.80478206e-02  1.85265783e-02  9.31689329e-03 -1.67358499e-02
  -1.16387736e-02  3.04822880e-03  3.56055386e-02  2.78010466e-05
   5.60677797e-02 -6.26950013e-03 -1.05933640e-02 -1.27088493e-02
  -3.85705046e-02 -9.82497931e-02  4.65339124e-02 -2.77524143e-02
   1.84488751e-03  4.14265580e-02  3.46447481e-03 -3.09899747e-02
   1.06126918e-02 -1.51194585e-02  6.88945502e-02  5.60522974e-02
  -5.55588715e-02 -9.00430884e-03 -2.10056342e-02 -4.43325005e-03
  -4.75944

In [ ]:
# Procurando nos registros do banco qual mais se assemelham à query
k = 10
D, I = index.search(query_embedding, k)  # Procura os top k  # noqa: E741
print(D, I)  # D = semelhança, I = índice

[[0.5567061  0.5478186  0.5435461  0.5373353  0.52883404 0.5111827
  0.491305   0.4742366  0.4421183  0.43844682]] [[ 9  3  2 10  0  1  4  8 19  5]]


In [ ]:
# Mostrando resultados iniciais
for semelhanca, indice in zip(D[0], I[0]):
    print(f"Item {indice} ({semelhanca:.3f}) - {info_animais[indice]}")

Item 9 (0.557) - Zebras são quadrúpedes com pelagem listrada em preto e branco, pelagem curta, cascos duros e vivem em bandos nas planícies africanas.
Item 3 (0.548) - Leopardos são felinos quadrúpedes ágeis com pelagem amarelada estampada por rosetas, arrastam presas para galhos de árvores e têm presas retráteis.
Item 2 (0.544) - Guepardos são quadrúpedes esguios com pelagem amarelada pontilhada de manchas, garras semi-retráteis e corpo adaptado à corrida em planícies abertas.
Item 10 (0.537) - Rinocerontes são quadrúpedes corpulentos com pele grossa e um ou dois chifres de queratina, pastam em savanas e florestas tropicais.
Item 0 (0.529) - Leões são quadrúpedes musculosos com pelagem curta e dourada, têm presas afiadas e caçam em grupos na savana africana.
Item 1 (0.511) - Tigres são felinos solitários e robustos, quadrúpedes de pelagem listrada e presas poderosas, que habitam florestas tropicais e pântanos.
Item 4 (0.491) - Gatos domésticos são quadrúpedes pequenos, de pelagens var

### Filtragem Híbrida
Utilizando os metadados para obter melhores resultados.

In [ ]:
# Filtrando somente aqueles que tem pelos
candidatos = []
for indice, semelhanca in zip(I[0], D[0]):
    metadados = metadados_animais[indice]
    if metadados["cobertura"] == "pelo":
        candidatos.append((indice, semelhanca))

In [ ]:
# Mostrando resultados após filtragem híbrida
for indice, semelhanca in candidatos:
    print(f"Item {indice} ({semelhanca:.3f}) - {info_animais[indice]}")

Item 9 (0.557) - Zebras são quadrúpedes com pelagem listrada em preto e branco, pelagem curta, cascos duros e vivem em bandos nas planícies africanas.
Item 3 (0.548) - Leopardos são felinos quadrúpedes ágeis com pelagem amarelada estampada por rosetas, arrastam presas para galhos de árvores e têm presas retráteis.
Item 2 (0.544) - Guepardos são quadrúpedes esguios com pelagem amarelada pontilhada de manchas, garras semi-retráteis e corpo adaptado à corrida em planícies abertas.
Item 0 (0.529) - Leões são quadrúpedes musculosos com pelagem curta e dourada, têm presas afiadas e caçam em grupos na savana africana.
Item 1 (0.511) - Tigres são felinos solitários e robustos, quadrúpedes de pelagem listrada e presas poderosas, que habitam florestas tropicais e pântanos.
Item 4 (0.491) - Gatos domésticos são quadrúpedes pequenos, de pelagens variadas e macias, com dentes caninos afiados e comportamento muitas vezes noturno.
Item 8 (0.474) - Girafas são quadrúpedes extremamente altos, com manch

### Re-rank

In [ ]:
# Criando os pares consulta/info do animal
pares = [[query, info_animais[indice]] for indice, semelhanca in candidatos]

In [ ]:
# Fazendo o cross-encoding
resultados_cross = cross_encoder.predict(pares)

In [ ]:
print(resultados_cross)  # Para cada índice nos candidatos, mostra o score

[-11.175043   -1.321177  -10.963656  -10.929853    0.7710285 -11.019899
 -11.021644  -11.062572 ]


In [ ]:
# Combinando índices, score do FAISS e score do Cross-Encoder
resultados = []
for (indice, faiss_score), cross_score in zip(candidatos, resultados_cross):
    resultados.append(
        {
            "indice": indice,
            "faiss": faiss_score,
            "cross": cross_score,
            "info": info_animais[indice],
        }
    )

In [ ]:
# Ordenando os resultados
resultados.sort(key=lambda x: x["cross"], reverse=True)  # por cross, decrescente

In [ ]:
# Mostrando o resultado final
print("Top 5 Felinos Após Re-Rank:")
for r in resultados[:5]:
    print(f"Item {r['indice']} ({r['cross']:.3f}) - {r['info']}")

Top 5 Felinos Após Re-Rank:
Item 1 (0.771) - Tigres são felinos solitários e robustos, quadrúpedes de pelagem listrada e presas poderosas, que habitam florestas tropicais e pântanos.
Item 3 (-1.321) - Leopardos são felinos quadrúpedes ágeis com pelagem amarelada estampada por rosetas, arrastam presas para galhos de árvores e têm presas retráteis.
Item 0 (-10.930) - Leões são quadrúpedes musculosos com pelagem curta e dourada, têm presas afiadas e caçam em grupos na savana africana.
Item 2 (-10.964) - Guepardos são quadrúpedes esguios com pelagem amarelada pontilhada de manchas, garras semi-retráteis e corpo adaptado à corrida em planícies abertas.
Item 4 (-11.020) - Gatos domésticos são quadrúpedes pequenos, de pelagens variadas e macias, com dentes caninos afiados e comportamento muitas vezes noturno.


Os top 5 foram exatamente os 5 felinos. :)

In [ ]:
!pip install faiss-cpu